# 🎨 Glimpse3D - SyncDreamer Inference

**Multi-view image generation using your local Glimpse3D project files**

## 🖥️ VS Code + Google Colab Setup

This notebook is designed to run via the **VS Code Colab Extension** with your local project files.

### Enable File Upload Feature:
1. **Open VS Code Settings** (`Ctrl+,`)
2. **Search for "Colab"**
3. **Enable the experimental "Uploading" setting**
4. **Reload VS Code** if prompted

### Upload Project Files:
Once enabled, right-click the **`ai_modules`** folder in the Explorer → **"Upload to Colab"**

> ⚠️ **Important:** Upload the entire `ai_modules` folder, NOT just `sync_dreamer`. This preserves the import path `from ai_modules.sync_dreamer import ...`

---

**Alternative:** The notebook will auto-clone from GitHub if local files aren't detected.

## 1️⃣ Check Environment & GPU

In [1]:
# Check if running on Colab and GPU availability
import sys
import os

# Check for Colab
IN_COLAB = 'google.colab' in sys.modules
print(f"Running in Colab: {IN_COLAB}")

# Check GPU
!nvidia-smi

import torch
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

## 2️⃣ Setup Project Files on Colab

**Option A: Use VS Code Colab Extension (Recommended)**
1. Enable experimental features in VS Code Settings → search "Colab" → enable **Uploading**
2. Right-click the **`ai_modules`** folder → **"Upload to Colab"**
3. Files will be uploaded to `/content/ai_modules/` on the Colab runtime

> ⚠️ Upload `ai_modules` (the parent folder), NOT `sync_dreamer` alone!

**Option B: Server Mounting (Advanced)**
- Use Command Palette: `Colab: Mount Server To Workspace`
- This lets you view/edit Colab files directly in VS Code!

**Option C: Auto-Clone from GitHub (Fallback)**
- If files aren't found, this cell will clone from the GitHub repository

In [ ]:
import os
import shutil
from pathlib import Path

# ============================================================
# DIRECTORY STRUCTURE EXPLANATION:
# ============================================================
# When you upload "ai_modules" folder via VS Code Colab extension,
# files go to: /content/ai_modules/sync_dreamer/
#
# When cloning from GitHub, files go to:
# /content/Glimpse-3D/ai_modules/sync_dreamer/
#
# The import path is: from ai_modules.sync_dreamer import ...
# So we need ai_modules/ to be in a directory that's on sys.path
# ============================================================

# Check possible locations for the ai_modules folder
possible_paths = [
    # VS Code "Upload to Colab" - uploads ai_modules to /content/
    Path("/content/ai_modules/sync_dreamer"),
    # Git clone location
    Path("/content/Glimpse-3D/ai_modules/sync_dreamer"),
]

print("🔍 Searching for ai_modules/sync_dreamer...")
FOUND_PATH = None
PROJECT_ROOT = None

for p in possible_paths:
    print(f"   Checking: {p}", end="")
    if p.exists() and (p / "inference.py").exists():
        FOUND_PATH = p
        # Set PROJECT_ROOT to parent of ai_modules
        PROJECT_ROOT = p.parent.parent  # sync_dreamer -> ai_modules -> project_root
        print(" ✅ FOUND!")
        break
    else:
        print(" ❌")

if FOUND_PATH is None:
    print("\n⚠️ ai_modules/sync_dreamer not found.")
    print("\n📋 To upload from VS Code:")
    print("   1. Right-click 'ai_modules' folder (NOT sync_dreamer)")
    print("   2. Select 'Upload to Colab'")
    print("   3. Re-run this cell")
    print("\n   Or clone from GitHub:")
    
    user_choice = input("\nClone from GitHub? (y/n): ").strip().lower()
    if user_choice == 'y':
        print("\n🔄 Cloning from GitHub repository...")
        !rm -rf /content/Glimpse-3D 2>/dev/null
        !git clone https://github.com/varunaditya27/Glimpse-3D.git /content/Glimpse-3D
        
        FOUND_PATH = Path("/content/Glimpse-3D/ai_modules/sync_dreamer")
        PROJECT_ROOT = Path("/content/Glimpse-3D")
        
        if FOUND_PATH.exists() and (FOUND_PATH / "inference.py").exists():
            print("✅ Successfully cloned!")
        else:
            print("❌ Clone failed or ai_modules/sync_dreamer missing in repo.")
            FOUND_PATH = None
    else:
        print("\n⏸️ Waiting for upload. Re-run this cell after uploading.")

# Set the SYNC_DREAMER_PATH variable for use in later cells
if FOUND_PATH:
    SYNC_DREAMER_PATH = FOUND_PATH
    print(f"\n✅ SYNC_DREAMER_PATH: {SYNC_DREAMER_PATH}")
    print(f"📁 PROJECT_ROOT: {PROJECT_ROOT}")
    print(f"\n📂 Module contents:")
    for item in sorted(SYNC_DREAMER_PATH.iterdir()):
        if item.is_dir():
            print(f"   📁 {item.name}/")
        else:
            size_kb = item.stat().st_size / 1024
            print(f"   📄 {item.name} ({size_kb:.1f} KB)")
else:
    print("\n❌ Cannot proceed without ai_modules/sync_dreamer. Please upload or clone first.")

## 3️⃣ Install Dependencies

In [3]:
# Install required packages
# Colab has PyTorch pre-installed - we just need the extras

!pip install -q omegaconf pytorch-lightning==1.9.0 einops kornia
!pip install -q transformers diffusers accelerate

# Install CLIP (required for image encoding)
!pip install -q git+https://github.com/openai/CLIP.git

# Install taming-transformers (rom1504 fork - required for VAE)
!pip install -q taming-transformers-rom1504

# Install image processing libraries
!pip install -q rembg[gpu] opencv-python-headless scikit-image imageio

# Verify taming is installed correctly
print("\n" + "="*50)
print("Verifying installations...")
print("="*50)

try:
    from taming.modules.vqvae.quantize import VectorQuantizer2
    print("✅ taming-transformers installed correctly!")
except ImportError as e:
    print(f"❌ taming import error: {e}")

try:
    import clip
    print("✅ CLIP installed correctly!")
except ImportError as e:
    print(f"❌ CLIP import error: {e}")

try:
    from rembg import remove
    print("✅ rembg installed correctly!")
except ImportError as e:
    print(f"❌ rembg import error: {e}")

## 4️⃣ Download Checkpoints (if needed)

Downloads two checkpoint files (~6GB total):
- `syncdreamer-pretrain.ckpt` (~5.2GB) - Main model weights
- `ViT-L-14.pt` (~890MB) - CLIP image encoder

In [4]:
import os
from pathlib import Path

# Checkpoint directory
CKPT_DIR = SYNC_DREAMER_PATH / "ckpt"
CKPT_DIR.mkdir(parents=True, exist_ok=True)

# Required checkpoints
CHECKPOINTS = {
    "syncdreamer-pretrain.ckpt": {
        "url": "https://huggingface.co/camenduru/SyncDreamer/resolve/main/syncdreamer-pretrain.ckpt",
        "size_gb": 5.2
    },
    "ViT-L-14.pt": {
        "url": "https://huggingface.co/camenduru/SyncDreamer/resolve/main/ViT-L-14.pt",
        "size_gb": 0.89
    }
}

# Install aria2 for faster downloads
!apt -y install -qq aria2

# Check and download each checkpoint
for filename, info in CHECKPOINTS.items():
    filepath = CKPT_DIR / filename
    
    if filepath.exists():
        size_gb = filepath.stat().st_size / (1024**3)
        if size_gb > info["size_gb"] * 0.9:  # At least 90% of expected size
            print(f"✅ {filename}: {size_gb:.2f} GB (already downloaded)")
            continue
        else:
            print(f"⚠️ {filename}: incomplete ({size_gb:.2f} GB), re-downloading...")
    
    print(f"📥 Downloading {filename} (~{info['size_gb']} GB)...")
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
        "{info['url']}" \
        -d "{CKPT_DIR}" -o "{filename}"
    
    # Verify download
    if filepath.exists():
        size_gb = filepath.stat().st_size / (1024**3)
        print(f"✅ {filename}: {size_gb:.2f} GB")
    else:
        print(f"❌ {filename}: Download failed!")

print("\n" + "="*50)
print(f"📁 Checkpoint directory: {CKPT_DIR}")
print("="*50)

## 5️⃣ Configure GPU Memory Settings

In [5]:
import os
import gc
import torch

# Set environment variables for memory optimization
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

# Enable memory-efficient settings
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

def print_gpu_memory():
    """Print current GPU memory usage"""
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1024**3
        reserved = torch.cuda.memory_reserved() / 1024**3
        total = torch.cuda.get_device_properties(0).total_memory / 1024**3
        print(f"GPU Memory: {allocated:.2f}GB allocated, {reserved:.2f}GB reserved, {total:.1f}GB total")

def clear_gpu_memory():
    """Clear GPU memory cache"""
    gc.collect()
    torch.cuda.empty_cache()
    print("🧹 GPU memory cache cleared")
    print_gpu_memory()

# Initial memory check
print_gpu_memory()

## 6️⃣ Load Glimpse3D SyncDreamer Module

This loads the `SyncDreamerService` from the Glimpse3D `ai_modules/sync_dreamer/` package.

In [6]:
import sys
from pathlib import Path

# Add PROJECT_ROOT to sys.path so Python can find ai_modules
# PROJECT_ROOT is the parent of ai_modules (e.g., /content or /content/Glimpse-3D)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
    print(f"✅ Added to sys.path: {PROJECT_ROOT}")

# Also add SYNC_DREAMER_PATH for ldm submodule imports
if str(SYNC_DREAMER_PATH) not in sys.path:
    sys.path.insert(0, str(SYNC_DREAMER_PATH))
    print(f"✅ Added to sys.path: {SYNC_DREAMER_PATH}")

print(f"\n📁 PROJECT_ROOT: {PROJECT_ROOT}")
print(f"📁 SYNC_DREAMER_PATH: {SYNC_DREAMER_PATH}")

# Import the Glimpse3D SyncDreamer module
print("\n🔄 Importing modules...")
try:
    from ai_modules.sync_dreamer import SyncDreamerService, generate_multiview
    from ai_modules.sync_dreamer.utils_syncdreamer import segment_foreground, preprocess_for_syncdreamer
    print("✅ Successfully imported from ai_modules.sync_dreamer!")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("\n🔄 Trying fallback direct import...")
    try:
        from inference import SyncDreamerService, generate_multiview
        from utils_syncdreamer import segment_foreground, preprocess_for_syncdreamer
        print("✅ Successfully imported via direct path!")
    except ImportError as e2:
        print(f"❌ Fallback also failed: {e2}")
        print("\n⚠️ Make sure you uploaded 'ai_modules' folder, not just 'sync_dreamer'!")

In [7]:
# Initialize the SyncDreamer service
print("🔄 Initializing SyncDreamer service...")

# Create service with paths to our checkpoints
service = SyncDreamerService(
    config_path=str(SYNC_DREAMER_PATH / "configs" / "syncdreamer.yaml"),
    checkpoint_path=str(SYNC_DREAMER_PATH / "ckpt" / "syncdreamer-pretrain.ckpt"),
    device="cuda"
)

# Load the model
print("\n🔄 Loading model weights (this may take a minute)...")
service.load_model()

print("\n" + "="*50)
print("✅ SyncDreamer model loaded and ready!")
print("="*50)
print_gpu_memory()

## 7️⃣ Prepare Test Image

You can:
1. Use a sample image from your project's assets
2. Upload your own image
3. Download a test image

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path

# Output directory for results
OUTPUT_DIR = Path("/content/output")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# =====================================================
# OPTION 1: Download a test image
# =====================================================
TEST_IMAGE_URL = "https://huggingface.co/spaces/One-2-3-45/One-2-3-45/resolve/main/demo_examples/01_astronaut.png"
TEST_IMAGE_PATH = Path("/content/test_input.png")

!wget -q "{TEST_IMAGE_URL}" -O "{TEST_IMAGE_PATH}"
print(f"📥 Downloaded test image to: {TEST_IMAGE_PATH}")

# =====================================================
# OPTION 2: Upload your own image (uncomment to use)
# =====================================================
# from google.colab import files
# uploaded = files.upload()
# TEST_IMAGE_PATH = Path(list(uploaded.keys())[0])
# print(f"📤 Uploaded: {TEST_IMAGE_PATH}")

# =====================================================
# OPTION 3: Use from project assets (uncomment to use)
# =====================================================
# TEST_IMAGE_PATH = PROJECT_ROOT / "assets" / "sample_inputs" / "your_image.png"

# Display the test image
print(f"\n📷 Using test image: {TEST_IMAGE_PATH}")
img = Image.open(TEST_IMAGE_PATH)

plt.figure(figsize=(6, 6))
plt.imshow(img)
plt.title(f"Input Image ({img.size[0]}x{img.size[1]}, {img.mode})")
plt.axis('off')
plt.show()

print(f"Image size: {img.size}")
print(f"Image mode: {img.mode}")

## 8️⃣ Preprocess Image (Background Removal)

SyncDreamer works best with images that have transparent backgrounds. We'll use `rembg` to remove the background if needed.

In [ ]:
import numpy as np
from PIL import Image

# Load image
input_img = Image.open(TEST_IMAGE_PATH)

# Check if image already has transparency
needs_segmentation = True
if input_img.mode == 'RGBA':
    alpha = np.array(input_img)[:, :, 3]
    if np.any(alpha < 255):
        print("✅ Image already has transparent background")
        needs_segmentation = False

# Remove background if needed
if needs_segmentation:
    print("🔄 Removing background with rembg...")
    processed_img = segment_foreground(input_img, method="rembg")
    print("✅ Background removed!")
else:
    processed_img = input_img

# Save processed image
PROCESSED_IMAGE_PATH = OUTPUT_DIR / "processed_input.png"
processed_img.save(PROCESSED_IMAGE_PATH)

# Display comparison
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

axes[0].imshow(input_img)
axes[0].set_title("Original")
axes[0].axis('off')

axes[1].imshow(processed_img)
axes[1].set_title("Processed (Background Removed)")
axes[1].axis('off')

plt.tight_layout()
plt.show()

print(f"\n📁 Processed image saved to: {PROCESSED_IMAGE_PATH}")

## 9️⃣ Run Multi-View Generation

**Key Parameters:**
- `elevation`: Input view elevation angle (0-40°, typically 30°)
- `crop_size`: Size to crop foreground to (default 200)
- `cfg_scale`: Classifier-free guidance scale (default 2.0)
- `batch_view_num`: Views per batch (4 for T4, 8 for A100)

In [ ]:
import time

# ============================================
# INFERENCE PARAMETERS
# ============================================
ELEVATION = 30.0        # Input view elevation (degrees)
CROP_SIZE = 200         # Foreground crop size (-1 to disable)
CFG_SCALE = 2.0         # Classifier-free guidance scale
BATCH_VIEW_NUM = 4      # 4 for T4 (15GB), 8 for A100
SAMPLE_NUM = 1          # Number of sample sets
SAMPLE_STEPS = 50       # DDIM sampling steps
SEED = 42               # Random seed

print("="*50)
print("🚀 Running Glimpse3D SyncDreamer Inference")
print("="*50)
print(f"📷 Input: {PROCESSED_IMAGE_PATH}")
print(f"📐 Elevation: {ELEVATION}°")
print(f"🎯 CFG Scale: {CFG_SCALE}")
print(f"📦 Batch View Num: {BATCH_VIEW_NUM}")
print(f"🔢 Sample Steps: {SAMPLE_STEPS}")
print(f"🎲 Seed: {SEED}")
print("="*50)

print_gpu_memory()

# Run inference using Glimpse3D service
print("\n⏳ Generating 16 multi-view images...")
start_time = time.time()

generated_views = service.generate(
    image=str(PROCESSED_IMAGE_PATH),
    elevation=ELEVATION,
    crop_size=CROP_SIZE,
    cfg_scale=CFG_SCALE,
    sample_num=SAMPLE_NUM,
    batch_view_num=BATCH_VIEW_NUM,
    sample_steps=SAMPLE_STEPS,
    seed=SEED
)

elapsed_time = time.time() - start_time
print(f"\n✅ Generation complete in {elapsed_time:.1f} seconds!")
print(f"📊 Generated {len(generated_views)} views")
print_gpu_memory()

## 🔟 Visualize Generated Views

The 16 views are arranged as:
- **Row 1-2**: Elevation 30° (views 0-7)
- **Row 3-4**: Elevation -20° (views 8-15)
- **Columns**: Azimuths 0°, 45°, 90°, 135°, 180°, 225°, 270°, 315°

In [ ]:
import matplotlib.pyplot as plt

# Get camera configuration from service
elevations = service.ELEVATIONS
azimuths = service.AZIMUTHS

# Create 4x4 grid visualization
fig, axes = plt.subplots(4, 4, figsize=(16, 16))

for i in range(16):
    row = i // 4
    col = i % 4
    axes[row, col].imshow(generated_views[i])
    axes[row, col].set_title(f"View {i}: E={elevations[i]}° A={azimuths[i]}°", fontsize=10)
    axes[row, col].axis('off')

plt.suptitle("Glimpse3D SyncDreamer - Generated Multi-View Images", fontsize=16, y=1.02)
plt.tight_layout()

# Save grid
grid_path = OUTPUT_DIR / "multiview_grid.png"
plt.savefig(grid_path, dpi=150, bbox_inches='tight')
plt.show()

print(f"📊 Grid saved to {grid_path}")

## 1️⃣1️⃣ Save Output Images

In [ ]:
import imageio
import numpy as np

# Save individual views
print("💾 Saving individual views...")
saved_paths = []

for i, view in enumerate(generated_views[:16]):
    filename = f"view_{i:02d}_elev{elevations[i]}_azim{azimuths[i]}.png"
    path = OUTPUT_DIR / filename
    view.save(path)
    saved_paths.append(path)

print(f"✅ Saved {len(saved_paths)} individual views to {OUTPUT_DIR}/")

# Save concatenated strip
concat_images = [np.array(v) for v in generated_views[:16]]
concat_strip = np.concatenate(concat_images, axis=1)
strip_path = OUTPUT_DIR / "concat_strip.png"
Image.fromarray(concat_strip).save(strip_path)
print(f"✅ Saved concatenated strip to {strip_path}")

# Create turntable GIF
print("🎬 Creating turntable animation...")
turntable_views = [np.array(generated_views[i]) for i in range(8)]  # First 8 views (30° elevation)
turntable_loop = turntable_views + turntable_views[::-1][1:-1]  # Forward + reverse for smooth loop

gif_path = OUTPUT_DIR / "turntable.gif"
imageio.mimsave(gif_path, turntable_loop, fps=4, loop=0)
print(f"✅ Saved turntable GIF to {gif_path}")

# Display GIF
from IPython.display import Image as IPImage, display
display(IPImage(filename=str(gif_path)))

# List all output files
print(f"\n📁 Output files in {OUTPUT_DIR}:")
for f in sorted(OUTPUT_DIR.iterdir()):
    size_kb = f.stat().st_size / 1024
    print(f"  - {f.name} ({size_kb:.1f} KB)")

## 📥 Download Results

In [ ]:
import shutil
from google.colab import files

# Create ZIP archive
zip_path = Path("/content/glimpse3d_syncdreamer_output")
shutil.make_archive(str(zip_path), 'zip', OUTPUT_DIR)
print(f"📦 Created {zip_path}.zip")

# Download
files.download(f"{zip_path}.zip")

## 🧹 Cleanup

In [ ]:
# Unload model to free GPU memory
service.unload_model()

# Or use the cleanup function
# from ai_modules.sync_dreamer import cleanup
# cleanup()

clear_gpu_memory()
print("✅ Cleanup complete!")

---

## 📝 Quick Reference: Using Glimpse3D SyncDreamer Module

### Method 1: Quick Function
```python
from ai_modules.sync_dreamer import generate_multiview

# Generate and save in one call
output_paths = generate_multiview(
    image_path="input.png",
    output_dir="outputs/",
    elevation=30.0,
    seed=42
)
```

### Method 2: Service Class (More Control)
```python
from ai_modules.sync_dreamer import SyncDreamerService

# Initialize and load
service = SyncDreamerService()
service.load_model()

# Generate images (returns PIL Images)
views = service.generate(
    image="input.png",
    elevation=30.0,
    batch_view_num=4  # Use 4 for T4, 8 for A100
)

# Or generate and save
paths = service.generate_and_save(
    image="input.png",
    output_dir="outputs/",
    save_grid=True
)

# Cleanup
service.unload_model()
```

### Method 3: Preprocessing Utilities
```python
from ai_modules.sync_dreamer.utils_syncdreamer import (
    segment_foreground,
    preprocess_for_syncdreamer
)

# Remove background
rgba_image = segment_foreground(image, method="rembg")

# Full preprocessing
processed = preprocess_for_syncdreamer(image, crop_size=200)
```

### Output Camera Configuration
SyncDreamer generates 16 views at fixed camera positions:
- Views 0-7: Elevation 30°, Azimuth 0°-315° (45° steps)
- Views 8-15: Elevation -20°, Azimuth 0°-315° (45° steps)

### Tips
- **Elevation**: Front-facing photos → 30°, Top-down → 60-80°
- **VRAM**: Use `batch_view_num=4` for T4 (15GB), `batch_view_num=8` for A100
- **Quality**: Try different `crop_size` values (150-200) for best results